# Wine Quality Analysis Exercise Solution

Adapted from Dipanjan Sarkar et al. 2018. [Practical Machine Learning with Python](https://link.springer.com/book/10.1007/978-1-4842-3207-1).

## Overview

This module focuses on building predictive models to predict wine quality (low, medium and high) based on other features, following the standard classification Machine Learning pipeline.

## Learning Objectives

- Build and evaluate predictive models for wine quality classification
- Apply and compare different machine learning algorithms:
  - Decision Trees
  - Random Forests
  - Extreme Gradient Boosting
- Interpret model results using:
  - Feature importance analysis
  - ROC curves
  - Decision surfaces
  - Partial dependence plots

### Tasks to complete

- Train and evaluate models using:
  - Decision Trees
  - Random Forests
  - XGBoost
- Generate model interpretations and visualizations
- Compare model performances

## Prerequisites

- Python programming environment
- Basic understanding of statistical and machine learning concepts
- Familiarity with common ML libraries


In [ ]:
# Import necessary dependencies
# We wil use matplotlib and seaborn for exploratory data analysis and visualizations
import os
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from graphviz import Source
from IPython.display import Image
from numpy import interp  # Use numpy.interp instead of scipy.interp
from skater.core.explanations import Interpretation
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer
from skater.model import InMemoryModel
from sklearn import metrics, tree
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize
from sklearn.tree import DecisionTreeClassifier


# Set matplotlib inline
%matplotlib inline

## Load and merge datasets


In [ ]:
# Define data paths (consider making this configurable)
white_wine_path = "../../Data/winequality-white.csv"
red_wine_path = "../../Data/winequality-red.csv"

# Load data, handling potential exceptions
try:
    white_wine = pd.read_csv(white_wine_path, sep=";")
    red_wine = pd.read_csv(red_wine_path, sep=";")
except FileNotFoundError:
    print(f"Error: Wine data files not found. Check paths: {white_wine_path}, {red_wine_path}")
    exit()  # Or handle the error differently

# Add wine type and quality label (combined for efficiency)
for wine, color in [(red_wine, "red"), (white_wine, "white")]:
    wine["wine_type"] = color

# bucket wine quality scores into qualitative quality labels
# Wine quality scores of 3, 4, and 5 are mapped to low quality,
# 6 and 7 are mapped to medium quality, 8 and 9 are mapped to high quality
# wines under the quality_label attribute.
red_wine["quality_label"] = red_wine["quality"].apply(
    lambda value: "low" if value <= 5 else "medium" if value <= 7 else "high"
)
red_wine["quality_label"] = pd.Categorical(
    red_wine["quality_label"], categories=["low", "medium", "high"]
)

white_wine["quality_label"] = white_wine["quality"].apply(
    lambda value: "low" if value <= 5 else "medium" if value <= 7 else "high"
)
white_wine["quality_label"] = pd.Categorical(
    white_wine["quality_label"], categories=["low", "medium", "high"]
)

# merge red and white wine datasets
wines = pd.concat([red_wine, white_wine])
# re-shuffle records just to randomize data points
wines = wines.sample(frac=1, random_state=42).reset_index(drop=True)

# merge red and white wine datasets
wines = pd.concat([red_wine, white_wine])
# re-shuffle records just to randomize data points
wines = wines.sample(frac=1, random_state=42).reset_index(drop=True)


### Understand dataset features and values


In [ ]:
print(white_wine.shape, red_wine.shape)
print(wines.info())

We have 4898 white wine data points and 1599 red wine data points. The
merged dataset contains a total of 6497 data points and we also get an idea of numeric and categorical
attributes.


In [ ]:
# Let’s take a peek at our dataset to see some sample data points.
wines.head()

## Utilty functions for model evaluation


In [ ]:
def get_metrics(true_labels, predicted_labels):
    print(
        "Accuracy:", np.round(metrics.accuracy_score(true_labels, predicted_labels), 4)
    )
    print(
        "Precision:",
        np.round(
            metrics.precision_score(true_labels, predicted_labels, average="weighted"),
            4,
        ),
    )
    print(
        "Recall:",
        np.round(
            metrics.recall_score(true_labels, predicted_labels, average="weighted"), 4
        ),
    )
    print(
        "F1 Score:",
        np.round(
            metrics.f1_score(true_labels, predicted_labels, average="weighted"), 4
        ),
    )


def display_classification_report(true_labels, predicted_labels, classes=None): # classes can be None
    report = metrics.classification_report(y_true=true_labels, y_pred=predicted_labels, labels=classes)
    print(report)


def display_confusion_matrix(true_labels, predicted_labels, classes=[1, 0]):
    total_classes = len(classes)
    level_labels = [total_classes * [0], list(range(total_classes))]
    cm = metrics.confusion_matrix(
        y_true=true_labels, y_pred=predicted_labels, labels=classes
    )
    cm_frame = pd.DataFrame(
        data=cm,
        columns=pd.MultiIndex(levels=[["Predicted:"], classes], codes=level_labels),
        index=pd.MultiIndex(levels=[["Actual:"], classes], codes=level_labels),
    )
    print(cm_frame)


def display_model_performance_metrics(true_labels, predicted_labels, classes=[1, 0]):
    print("Model Performance metrics:")
    print("-" * 30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print("\nModel Classification report:")
    print("-" * 30)
    display_classification_report(
        true_labels=true_labels, predicted_labels=predicted_labels, classes=classes
    )
    print("\nPrediction Confusion Matrix:")
    print("-" * 30)
    display_confusion_matrix(
        true_labels=true_labels, predicted_labels=predicted_labels, classes=classes
    )

def plot_model_roc_curve(
    clf, features, true_labels, label_encoder=None, class_names=None
):
    ## Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    if hasattr(clf, "classes_"):
        class_labels = clf.classes_
    elif label_encoder:
        class_labels = label_encoder.classes_
    elif class_names:
        class_labels = class_names
    else:
        raise ValueError(
            "Unable to derive prediction classes, please specify class_names!"
        )
    n_classes = len(np.unique(true_labels)) # directly get the number of classes
    y_test = label_binarize(true_labels, classes=np.unique(true_labels)) # binarize based on actual classes
    if n_classes == 2:
        if hasattr(clf, "predict_proba"):
            prob = clf.predict_proba(features)
            y_score = prob[:, prob.shape[1] - 1]
        elif hasattr(clf, "decision_function"):
            prob = clf.decision_function(features)
            y_score = prob[:, prob.shape[1] - 1]
        else:
            raise AttributeError(
                "Estimator doesn't have a probability or confidence scoring system!"
            )

        fpr, tpr, _ = roc_curve(y_test, y_score)
        roc_auc = auc(fpr, tpr)
        plt.plot(
            fpr,
            tpr,
            label="ROC curve (area = {0:0.2f})" "".format(roc_auc),
            linewidth=2.5,
        )

    elif n_classes > 2:
        if hasattr(clf, "predict_proba"):
            y_score = clf.predict_proba(features)
        elif hasattr(clf, "decision_function"):
            y_score = clf.decision_function(features)
        else:
            raise AttributeError(
                "Estimator doesn't have a probability or confidence scoring system!"
            )

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        ## Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        ## Compute macro-average ROC curve and ROC area
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])  # Use numpy.interp instead of scipy.interp
        # Finally average it and compute AUC
        mean_tpr /= n_classes
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

        ## Plot ROC curves
        plt.figure(figsize=(6, 4))
        plt.plot(
            fpr["micro"],
            tpr["micro"],
            label="micro-average ROC curve (area = {0:0.2f})" "".format(
                roc_auc["micro"]
            ),
            linewidth=3,
        )

        plt.plot(
            fpr["macro"],
            tpr["macro"],
            label="macro-average ROC curve (area = {0:0.2f})" "".format(
                roc_auc["macro"]
            ),
            linewidth=3,
        )

        for i, label in enumerate(class_labels):
            plt.plot(
                fpr[i],
                tpr[i],
                label="ROC curve of class {0} (area = {1:0.2f})" "".format(
                    label, roc_auc[i]
                ),
                linewidth=2,
                linestyle=":",
            )
    else:
        raise ValueError("Number of classes should be atleast 2 or more")

    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower right")
    plt.show()




def plot_model_decision_surface(
    clf,
    train_features,
    train_labels,
    plot_step=0.02,
    cmap=plt.cm.RdYlBu,
    markers=None,
    alphas=None,
    colors=None,
):
    if train_features.shape[1] != 2:
        raise ValueError("X_train should have exactly 2 columnns!")

    x_min, x_max = (
        train_features[:, 0].min() - plot_step,
        train_features[:, 0].max() + plot_step,
    )
    y_min, y_max = (
        train_features[:, 1].min() - plot_step,
        train_features[:, 1].max() + plot_step,
    )
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, plot_step), np.arange(y_min, y_max, plot_step)
    )

    clf_est = clone(clf)
    clf_est.fit(train_features, train_labels)
    if hasattr(clf_est, "predict_proba"):
        Z = clf_est.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    else:
        Z = clf_est.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=cmap)

    le = LabelEncoder()
    y_enc = le.fit_transform(train_labels)
    n_classes = len(le.classes_)
    plot_colors = "".join(colors) if colors else [None] * n_classes
    label_names = le.classes_
    markers = markers if markers else [None] * n_classes
    alphas = alphas if alphas else [None] * n_classes
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y_enc == i)
        plt.scatter(
            train_features[idx, 0],
            train_features[idx, 1],
            c=color,
            label=label_names[i],
            cmap=cmap,
            edgecolors="black",
            marker=markers[i],
            alpha=alphas[i],
        )
    plt.legend()
    plt.show()

## Predicting Wine Quality

We will predict the wine quality ratings based on other features.

### Prepare features

#### Feature selection

To start with, we
will first select our necessary features and separate out the prediction class labels and prepare train and test
datasets. We use the prefix **wqp\_** in our variables to easily identify them as needed, where **wqp** depicts wine
quality prediction.


In [ ]:
wqp_features = wines.iloc[:, :-3]
wqp_class_labels = wines["quality_label"] # Directly use the series
wqp_feature_names = list(wqp_features.columns)

wqp_train_X, wqp_test_X, wqp_train_y, wqp_test_y = train_test_split(wqp_features, wqp_class_labels, test_size=0.3, random_state=42, stratify=wqp_class_labels) # Add stratification

print(Counter(wqp_train_y), Counter(wqp_test_y))
print("Features:", wqp_feature_names)

The numbers show us the wine samples for each class and we can also see the feature names which will
be used in our feature set.


#### Feature Scaling

We will be using a standard scaler in this
scenario.


In [ ]:
# Define the scaler
wqp_ss = StandardScaler()

# Scale the train set
wqp_train_SX = wqp_ss.fit_transform(wqp_train_X)

# Scale the test set
wqp_test_SX = wqp_ss.transform(wqp_test_X)

### Train, Predict & Evaluate Model using Decision Tree

The main advantage of decision tree based models is model
interpretability, since it is quite easy to understand and interpret the decision rules which led to a specific
model prediction. Besides this, other advantages include the model’s ability to handle both categorical
and numeric data with ease as well as multi-class classification problems. Trees can be even visualized to
understand and interpret decision rules better.


#### Train the model using DecisionTreeClassifier


In [ ]:
wqp_dt = DecisionTreeClassifier()
wqp_dt.fit(wqp_train_SX, wqp_train_y)

#### Evaluate model performance


In [ ]:
# let’s predict the wine types for our test data samples and evaluate the performance.
wqp_dt_predictions = wqp_dt.predict(wqp_test_SX)

display_model_performance_metrics(
    true_labels=wqp_test_y, predicted_labels=wqp_dt_predictions, classes=wqp_label_names
)

We get an overall F1 Score and model accuracy of approximately 72%.

Looking at the class based statistics; we can see the recall for the high quality
wine samples is pretty bad since a lot of them have been misclassified into medium and low quality ratings.
This is kind of expected since we do not have a lot of training samples for high quality wine if you remember
our training sample sizes from earlier. Considering low and high quality rated wine samples, we should at
least try to see if we can prevent our model from predicting a low quality wine as high and similarly prevent
predicting a high quality wine as low.


#### Model Interpretation


##### Visualize Feature Importances from Decision Tree Model


In [ ]:
wqp_dt_feature_importances = wqp_dt.feature_importances_
wqp_dt_feature_names, wqp_dt_feature_scores = zip(
    *sorted(zip(wqp_feature_names, wqp_dt_feature_importances), key=lambda x: x[1])
)
y_position = list(range(len(wqp_dt_feature_names)))
plt.barh(y_position, wqp_dt_feature_scores, height=0.6, align="center")
plt.yticks(y_position, wqp_dt_feature_names)
plt.xlabel("Relative Importance Score")
plt.ylabel("Feature")
t = plt.title("Feature Importances for Decision Tree")

We can clearly observe that the most important features have changed as compared to
our previous model. _Alcohol_ and _volatile acidity_ occupy the top two ranks and _total sulfur dioxide_
seems to be one of the most important features for classifying both wine type and quality.


##### Visualize the Decision Tree


In [ ]:
graph = Source(
    tree.export_graphviz(
        wqp_dt,
        out_file=None,
        class_names=wqp_label_names,
        filled=True,
        rounded=True,
        special_characters=False,
        feature_names=wqp_feature_names,
        max_depth=3,
    )
)
png_data = graph.pipe(format="png")
with open("dtree_structure.png", "wb") as f:
    f.write(png_data)

Image(png_data)

Our decision tree model has a huge number of nodes and branches hence we visualized our tree for a
max depth of three.

You can start observing the decision rules from the tree
in the figure where the starting split is determined by the rule of alcohol <= -0.128 and with each
yes\no decision branch split, we have further decision nodes as we descend into the tree at each depth level.
The class variable is what we are trying to predict, i.e. wine quality being low, medium, or high and value
determines the total number of samples at each class present in the current decision node at each instance.

The gini parameter is basically the criterion which is used to determine and measure the quality of the split
at each decision node. Best splits can be determined by metrics like gini impurity\gini index or information
gain, a metric that helps in minimizing the probability of
misclassification.


### Train, Predict & Evaluate Model using Random Forests

In the random
forest model, each base learner is a decision tree model trained on a bootstrap sample of the training data.
Besides this, when we want to split a decision node in the tree, the split is chosen from a random subset of all
the features instead of taking the best split from all the features.


#### Train the model using RandomForestClassifier


In [ ]:
# train the model
wqp_rf = # Your code goes here
wqp_rf.fit(# Your code goes here)

#### Evaluate model performance


In [ ]:
# predict and evaluate performance
wqp_rf_predictions = # Your code goes here
display_model_performance_metrics(
    true_labels=wqp_test_y, predicted_labels=wqp_rf_predictions, classes=wqp_label_names
)

The model prediction results on the test dataset depict an overall F1 Score and model accuracy of
approximately 80%. This is definitely an improvement of 7% from what we obtained
with just decision trees proving that ensemble learning is working better.


#### Hyperparameter tuning with Grid Search & Cross Validation

Another way to further improve on this result is model tuning. To be more specific, models have
hyperparameters that can be tuned.

Hyperparameters are also known as meta-parameters
and are usually set before we start the model training process. These hyperparameters do not have any
dependency on being derived from the underlying data on which the model is trained. Usually these
hyperparameters represent some high level concepts or knobs, which can be used to tweak and tune the
model during training to improve its performance. Our random forest model has several hyperparameters as shown below.


In [ ]:
print(wqp_rf.get_params())

##### Get the best hyperparameter values using Grid Search

TODO: this fit gives quite a few warnings/errors.


In [ ]:
# Optimized Grid Search for Random Forest
param_grid = {
    "n_estimators": [100, 200, 500], # Removed 300
    "max_features": ["sqrt", "log2"], # Removed None, often not beneficial
    "min_samples_split": [2, 5, 10],   # Added regularization parameters
    "min_samples_leaf": [1, 2, 4]
}

wqp_clf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=1) # n_jobs for parallel processing
wqp_clf.fit(wqp_train_SX, wqp_train_y)
print(wqp_clf.best_params_)

We have 500 estimators and auto maximum features which represents the square root of the total
number of features to be considered during the best split operations.


##### View grid search results


In [ ]:
results = wqp_clf.cv_results_
for param, score_mean, score_sd in zip(
    results["params"], results["mean_test_score"], results["std_test_score"]
):
    print(param, round(score_mean, 4), round(score_sd, 4))

The output shows the selected hyperparameter combinations and its corresponding mean
accuracy and standard deviation values across the grid.


### Train, Predict & Evaluate Random Forest Model with tuned hyperparameters


In [ ]:
wqp_rf = RandomForestClassifier(# Your code goes here)
wqp_rf.fit(wqp_train_SX, wqp_train_y)

wqp_rf_predictions = wqp_rf.predict(wqp_test_SX)
display_model_performance_metrics(
    true_labels=wqp_test_y, predicted_labels=wqp_rf_predictions, classes=wqp_label_names
)

The model prediction results on the test dataset improved the overall F1 Score and model accuracy a little bit from the initial random forest model.


## Train, Predict & Evaluate Model using Extreme Gradient Boosting

Another way of modeling ensemble based methods is boosting. A very popular method is XGBoost
which stands for Extreme Gradient Boosting. It is a variant of the Gradient Boosting Machines (GBM)
model. This model is extremely popular in the Data Science community owing to its superior performance
in several Data Science challenges and competitions especially on Kaggle.


### Load and set dependencies


In [ ]:
# initialize and fit LabelEncoder
label_encoder = LabelEncoder()
wqp_train_y_encoded = # Your code goes here
wqp_test_y_encoded = label_encoder.transform(wqp_test_y) # Encode test labels as well

### Train the model


In [ ]:
wqp_xgb_model = xgb.XGBClassifier(seed=42, eval_metric='mlogloss') # use_label_encoder=False and eval_metric for multiclass
wqp_xgb_model.fit(wqp_train_SX, wqp_train_y_encoded)

### Predict and Evaluate Model


In [ ]:
# Predict and Evaluate Model
wqp_xgb_predictions = wqp_xgb_model.predict(wqp_test_SX)

# Decode the predictions back to string labels
wqp_xgb_predictions_decoded = label_encoder.inverse_transform(wqp_xgb_predictions)

# Display model performance metrics
display_model_performance_metrics(
    true_labels=wqp_test_y,
    predicted_labels=wqp_xgb_predictions_decoded,  # Use decoded predictions
    classes=wqp_label_names,
)

### Tuning hyperparameters

#### Get the best hyperparameter values


In [ ]:
# Optimized Grid Search for XGBoost
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 7, 10], # Removed 15
    "learning_rate": [0.1, 0.3], # Added 0.1
    "gamma": [0, 0.1, 0.2],       # Added regularization
    "subsample": [0.8, 1.0],     # Added subsampling
    "colsample_bytree": [0.8, 1.0] # Added column sampling
}

wqp_clf = GridSearchCV(xgb.XGBClassifier(seed=42, eval_metric='mlogloss'), param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=1)
wqp_clf.fit(wqp_train_SX, wqp_train_y_encoded)
print(wqp_clf.best_params_)

#### View grid search results


In [ ]:
results = wqp_clf.cv_results_
for param, score_mean, score_sd in zip(
    results["params"], results["mean_test_score"], results["std_test_score"]
):
    print(param, round(score_mean, 4), round(score_sd, 4))

### Train, Predict & Evaluate Extreme Gradient Boosted Model with tuned hyperparameters


In [ ]:
# Train the XGBoost model with the best hyperparameters
wqp_xgb_model = xgb.XGBClassifier(
    seed=42, # Your code goes here
)
wqp_xgb_model.fit(wqp_train_SX, wqp_train_y_encoded)  # Use encoded labels here

# Predict on the test set
wqp_xgb_predictions = wqp_xgb_model.predict(wqp_test_SX)

# Decode the predictions back to string labels
wqp_xgb_predictions_decoded = label_encoder.inverse_transform(wqp_xgb_predictions)

# Display model performance metrics
display_model_performance_metrics(
    true_labels=wqp_test_y,
    predicted_labels=wqp_xgb_predictions_decoded,  # Use decoded predictions
    classes=wqp_label_names,
)


The model prediction results on the test dataset depict an overall F1 Score and model accuracy of
approximately 78%. Though random forests perform slightly better, it definitely
performs better than a basic model like a decision tree.


## Model Interpretation


### Comparative analysis of Model Feature importances


In [ ]:
# leveraging skater for feature importances
interpreter = Interpretation(wqp_test_SX, feature_names=wqp_feature_names)
wqp_im_model = InMemoryModel(
    wqp_rf.predict_proba, examples=wqp_train_SX, target_names=wqp_rf.classes_
)
# retrieving feature importances from the scikit-learn estimator
wqp_rf_feature_importances = wqp_rf.feature_importances_
wqp_rf_feature_names, wqp_rf_feature_scores = zip(
    *sorted(zip(wqp_feature_names, wqp_rf_feature_importances), key=lambda x: x[1])
)
# plot the feature importance plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
t = f.suptitle("Feature Importances for Random Forest", fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.6)
y_position = list(range(len(wqp_rf_feature_names)))
ax1.barh(
    y_position,
    wqp_rf_feature_scores,
    height=0.6,
    align="center",
    tick_label=wqp_rf_feature_names,
)
ax1.set_title("Scikit-Learn")
ax1.set_xlabel("Relative Importance Score")
ax1.set_ylabel("Feature")
plots = interpreter.feature_importance.plot_feature_importance(
    wqp_im_model, ascending=True, ax=ax2
)
ax2.set_title("Skater")
ax2.set_xlabel("Relative Importance Score")
ax2.set_ylabel("Feature")

We can clearly observe from Figure that the most important features are consistent across the
two plots, which is expected considering we are just using different interfaces on the same model. The


### Visualize Model ROC Curve


In [ ]:
plot_model_roc_curve(wqp_rf, wqp_test_SX, wqp_test_y)

The AUC is pretty good based on what we see. The dotted lines indicate the per-class ROC curves and
the lines in bold are the macro and micro-average ROC curves.


### Visualize Model Decision Surface


In [ ]:
feature_indices = [
    i
    for i, feature in enumerate(wqp_feature_names)
    if feature in ["alcohol", "volatile acidity"]
]
plot_model_decision_surface(
    clf=wqp_rf,
    train_features=wqp_train_SX[:, feature_indices],
    train_labels=wqp_train_y,
    plot_step=0.02,
    cmap=plt.cm.RdYlBu,
    markers=[",", "d", "+"],
    alphas=[1.0, 0.8, 0.5],
    colors=["r", "b", "y"],
)

### Interpreting Model Predictions


In [ ]:
exp = LimeTabularExplainer(
    wqp_train_SX,
    feature_names=wqp_feature_names,
    discretize_continuous=True,
    class_names=wqp_rf.classes_,
)
# Model interpretation for our wine quality model's prediction for a low quality wine
exp.explain_instance(
    wqp_test_SX[10], wqp_rf.predict_proba, top_labels=1
).show_in_notebook()

The figure above shows us the features that were primarily responsible for the model to
predict the wine quality as low. We can see that the most important feature was alcohol, which makes sense
considering what we obtained in our analyses so far from feature importances and model decision surface
interpretations. The values for each corresponding feature depicted here are the scaled values obtained after
feature scaling.


In [ ]:
# Model interpretation for our wine quality model's prediction for a high quality wine
# Your code goes here

We can see the features responsible for the model correctly
predicting the wine quality as high and the primary feature was again alcohol by volume (besides other
features like density, volatile acidity, and so on). Also you can notice a stark difference in the scaled
values of alcohol for the two instances


#### Visualizing Partial Dependencies

In general, partial dependence helps describe the marginal impact
or influence of a feature on the model prediction decision by holding the other features constant. Because it
is very difficult to visualize high dimensional feature spaces, typically one or two influential and important
features are used to visualize partial dependence plots.


##### One-way partial dependence


In [ ]:
# plot one-way partial dependence plots for our model prediction function
# based on the most important feature, alcohol.
axes_list = interpreter.partial_dependence.plot_partial_dependence(
    ["alcohol"], wqp_im_model, grid_resolution=100, with_variance=True, figsize=(6, 4)
)
axs = axes_list[0][3:]
[ax.set_ylim(0, 1) for ax in axs]

We can see that with an increase in the quantity of alcohol content, the
confidence\probability of the model predictor increases in predicting the wine to be either medium or high
and similarly it decreases for the probability of wine to be of low quality. This shows there is definitely some
relationship between the class predictions with the alcohol content and again the influence of alcohol for
predictions of class high is pretty low, which is expected considering training samples for high quality wine
are less.


##### Two-way partial dependence


In [ ]:
import warnings
from numpy import VisibleDeprecationWarning
warnings.filterwarnings("ignore", category=VisibleDeprecationWarning)

# plot wwo-way partial dependence plots for our random forest model predictor
# based on alcohol and volatile acidity
plots_list = # Your code goes here
axs = plots_list[0][3:]
[ax.set_zlim(0, 1) for ax in axs]

We can see predicting high
quality wine, due to the lack of training data, while some dependency is there for high wine quality class
prediction with the increase in alcohol and corresponding decrease in volatile acidity is it quite weak, as
we can see in the left most plot. There also seems to be a strong dependency on low wine quality class
prediction with the corresponding decrease in alcohol and the increase in volatile acidity levels. This
is clearly visible in the rightmost plot. The plot in the middle talks about medium wine quality class
predictions. We can observe predictions having a strong dependency with corresponding increase in alcohol
and with decrease in volatile acidity levels. This should give you a good foundation on leveraging partial
dependence plots to dive deeper into model interpretation.


## Conclusion

Through this module, we covered:

- Building and evaluating multiple classification models
- Applying different machine learning algorithms and comparing their performance
- Using model interpretation techniques including:
  - Feature importance analysis
  - ROC curves and performance metrics
  - Decision surface visualization
  - Partial dependence plots
- Understanding how different features influence model predictions

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.
